In [2]:
import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
from pdf2image import convert_from_path
import PIL
from PIL import Image, ImageFilter
from wand.image import Image as Img
import os
import os.path
from PIL import Image
import pyocr
import pyocr.builders
from PIL import Image
tools = pyocr.get_available_tools()[0]
# The tools are returned in the recommended order of usage

langs = tools.get_available_languages()
lang = langs[0]


In [3]:
pdfFileObj = open('filled_koa-ca-1.pdf', 'rb') 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
print(pdfReader.numPages) 

25


In [4]:
pageObj = pdfReader.getPage(0) 

In [5]:
print(pageObj.cropBox.getLowerLeft())
print(pageObj.cropBox.getUpperLeft())
print(pageObj.cropBox.getUpperRight())
print(pageObj.cropBox.getLowerRight())

(0, 0)
(0, 792)
(612, 792)
(612, 0)


In [6]:
72*(2.84)

204.48

In [7]:
## Got the first row
## need to turn image into pixels

In [11]:
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)

    bottomMargin = 63
    sideMargin = 56
    horizantalSpacing = 11
    verticalSpacing = 47

    writer = PdfFileWriter()

    questions = 5
    parts = 6
    answers = [[1,2,4],[1,3,4],[2,3,5],[1,4,5],[0,3,5]]

    pageObj.cropBox.lowerLeft = (29 , 740)
    pageObj.cropBox.upperRight = (32 + 64 , 745 + 26)
    fileName = "identifier.pdf"
    writer.addPage(pageObj)
    outstream = open(fileName,'wb')
    writer.write(outstream)
    outstream.close()
    with Img(filename= "identifier.pdf", resolution=300) as img:   # convert pdf to jpg
        img.compression_quality = 99 
        img.save(filename= "identifier.jpg")
    img = Image.open('identifier.jpg')
    img = img.convert('L')
    img.save('identifier.jpg')

    bashCommand = "convert identifier.jpg -type Grayscale -negate -define morphology:compose=darken -morphology Thinning 'Rectangle:1x80+0+0<' -negate out.png"
    import subprocess
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

    worksheetName = tools.image_to_string(Image.open('out.png'), 
                                 builder=pyocr.builders.DigitBuilder())
    print(worksheetName)
    if not os.path.exists(worksheetName):
        os.makedirs(worksheetName)

    pageObj = pdfReader.getPage(i)
    writer = PdfFileWriter()

    pageObj.cropBox.lowerLeft = (200 , 739)
    pageObj.cropBox.upperRight = (282 , 745 + 30)
    fileName = "student.pdf"
    writer.addPage(pageObj)
    outstream = open(fileName,'wb')
    writer.write(outstream)
    outstream.close()

    with Img(filename= "student.pdf", resolution=300) as img:   # convert pdf to jpg
        img.compression_quality = 99 
        img.save(filename= "students.jpg")
    img = Image.open('students.jpg')
    img = img.convert('L')
    img.save('students.jpg')

#bashCommand = "convert students.jpg -type Grayscale -negate -define morphology:compose=darken -morphology Thinning 'Rectangle:1x80+0+0<' -negate studentsout.png"
#process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
#output, error = process.communicate()

    studentName = tools.image_to_string(Image.open('students.jpg'), 
                                 builder=pyocr.builders.DigitBuilder())
    print(studentName)
    
    pathName = worksheetName + "/" + studentName
    print(pathName)
    if not os.path.exists(pathName):
        os.makedirs(pathName)
    

"""    
for i in range(questions, 0, -1):
    for j in range(parts):
        if j not in answers[i - 1]:
            continue
        else:
            
            fileName = "image_crop_" + "Q" + str(i) + "_P" + str(j)
            #print(fileName)

            leftSide = sideMargin + (horizantalSpacing * j) + (72 * j)
            bottomSide = bottomMargin + ((5 - i) * 72) + ((5 - i) * verticalSpacing)

            pageObj.cropBox.lowerLeft = (leftSide , bottomSide)
            pageObj.cropBox.upperRight = (leftSide + 72 , bottomSide + 72)
            writer.addPage(pageObj)
            outstream = open(fileName + ".pdf",'wb')
            writer.write(outstream)
            outstream.close()
            with Img(filename= fileName + ".pdf", resolution=300) as img:   # convert pdf to jpg
                img.compression_quality = 99 
                img.save(filename= fileName + ".jpg")
                os.remove(fileName + ".pdf")  #remove the pdf file
"""

Koa-CA-1
Koa-SLO-19
Koa-CA-1/Koa-SLO-19
Koa-CA-1
Koa-SLO-18
Koa-CA-1/Koa-SLO-18
Koa-CA-1
Koa-SLO-17
Koa-CA-1/Koa-SLO-17
Koa-CA-1
Koa-SLO-16
Koa-CA-1/Koa-SLO-16
Koa-CA-1
Koa-SLO-15
Koa-CA-1/Koa-SLO-15
Koa-CA-1
Koa-SLO-14
Koa-CA-1/Koa-SLO-14
Koa-CA-1
Koa-SLO-13
Koa-CA-1/Koa-SLO-13
Koa-CA-1
Koa-SLO-12
Koa-CA-1/Koa-SLO-12
Koa-CA-1
Koa-SLO-11
Koa-CA-1/Koa-SLO-11
Koa-CA-1
Koa-SLO-10
Koa-CA-1/Koa-SLO-10
Koa-CA-1
Koa-SLO-1
Koa-CA-1/Koa-SLO-1
Koa-CA-1
Koa-SLO-0
Koa-CA-1/Koa-SLO-0
Koa-CA-1
Koa-SLO-9
Koa-CA-1/Koa-SLO-9
Koa-CA-1
Koa-SLO-8
Koa-CA-1/Koa-SLO-8
Koa-CA-1
Koa-SLO-7
Koa-CA-1/Koa-SLO-7
Koa-CA-1
Koa-SLO-6
Koa-CA-1/Koa-SLO-6
Koa-CA-1
Koa-SLO-5
Koa-CA-1/Koa-SLO-5
Koa-CA-1
Koa-SLO-4
Koa-CA-1/Koa-SLO-4
Koa-CA-1
Koa-SLO-3
Koa-CA-1/Koa-SLO-3
Koa-CA-1
Koa-SLO-24
Koa-CA-1/Koa-SLO-24
Koa-CA-1
Koa-SLO-23
Koa-CA-1/Koa-SLO-23
Koa-CA-1
Koa-SLO-22
Koa-CA-1/Koa-SLO-22
Koa-CA-1
Koa-SLO-21
Koa-CA-1/Koa-SLO-21
Koa-CA-1
Koa-SLO-20
Koa-CA-1/Koa-SLO-20
Koa-CA-1
Koa-SLO-2
Koa-CA-1/Koa-SLO-2


'    \nfor i in range(questions, 0, -1):\n    for j in range(parts):\n        if j not in answers[i - 1]:\n            continue\n        else:\n            \n            fileName = "image_crop_" + "Q" + str(i) + "_P" + str(j)\n            #print(fileName)\n\n            leftSide = sideMargin + (horizantalSpacing * j) + (72 * j)\n            bottomSide = bottomMargin + ((5 - i) * 72) + ((5 - i) * verticalSpacing)\n\n            pageObj.cropBox.lowerLeft = (leftSide , bottomSide)\n            pageObj.cropBox.upperRight = (leftSide + 72 , bottomSide + 72)\n            writer.addPage(pageObj)\n            outstream = open(fileName + ".pdf",\'wb\')\n            writer.write(outstream)\n            outstream.close()\n            with Img(filename= fileName + ".pdf", resolution=300) as img:   # convert pdf to jpg\n                img.compression_quality = 99 \n                img.save(filename= fileName + ".jpg")\n                os.remove(fileName + ".pdf")  #remove the pdf file\n'

In [ ]:
outstream = open(fileName,'wb')
writer.write(outstream)
outstream.close()

In [8]:
def get_pixel(image, i, j):
    # Inside image bounds?
    width, height = image.size
    if i > width or j > height:
        return None

    # Get Pixel
    pixel = image.getpixel((i, j))
    return pixel

In [ ]:
im = Image.open('A.png', 'r')
pix_val = list(im.getdata())
pix_val_flat = [x for sets in pix_val for x in sets]

In [ ]:
im.size

In [9]:
def imageprepare(argv):
    
    im = Image.open(argv).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28,28), (255)) ## Create white canvas of 28x28
    
    
    if width > height:
        nheight = int(round((20.0 / width * height), 0))
        if (nheight == 0):
            nheight = 1
            
        img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight)/ 2),0))
        newImage.paste(img, (4, wtop))
    else:   
        nwidth = int(round((20.0 / height * width), 0))
        if(nwidth == 0):
            nwidth = 1
        img = im.resize((nwidth,20),Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth) / 2), 0))
        newImage.paste(img, (wleft,4))

    tv = list(newImage.getdata())
    
    tva = [(255 - x)*1.0 / 255.0 for x in tv]
  
    return tva

In [ ]:
d = imageprepare('A.png')

In [ ]:
https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()